# Simulate core energy metabolism

Computationally assess the characteristics of a simple model of core energy metabolism that includes:

1. Glycolysis
2. Pentose phosphate pathway
3. TCA Cycle
4. Electron transport chain

In [5]:
from __future__ import print_function
import cobra
from matplotlib import pyplot as plt
import escher
import os
import pandas as pd
import numpy as np
from beng213 import resources

from os.path import dirname, abspath

resource_dir = '../resources/'

%matplotlib inline

In [6]:
# The reactions and metabolites to be added in this exercise
builder = escher.Builder(map_json='%s/gly_ppp_tca_etc_map.json'% resource_dir)
builder

Builder()

In [8]:
model_cons = cobra.io.load_json_model('%s/glycolysis_ppp_tca_etc_model.json' % resource_dir)

## A) Revisiting old assumptions
### 1) Set the objective to ATPM and optimize

In [9]:
# start each section by copying model to prevent losing track of model changes
model = model_cons.copy()

### 2) Assess the cause of the decrease in ATP yield
Using the glycolysis model and the `pyr_to_ATP` and `NADH_to_ATP` pseudo-reactions, the computed yield was 32 ATP per glucose

Use the `h_i` metabolite's `summary` method to find the reactions producing and consumine the high energy protons

## B) Modeling alternative pathways

### 1) Characterize the consequence of the two "extreme" fates for glucose 6 phosphate

Either all flux through upper glycolysis (PGI) or all flux through PPP (G6PDH2r)

Knock out PGI and optimize for ATPM flux. Display the fluxes through the following reactions on a bar chart:

| ID      | Interpretation 
| :-------------: |:-------------:|
| ATPM | ATP production potential |
| GTHOr | NADPH produced | 
| EX_h_c | Protons produced    | 
| EX_co2_c | CO$_2$ produced     | 
| EX_h2o_c | Water produced     | 

Repeat analysis with G6PDH2r knocked out

### 2) Characterizing the solution space: Flux variability analysis (FVA)

Maximize and minimize the flux through G6PDH2r and PGI for values of ATPM form 0 to the maximum values
 - **Note** minimization is performed by passing 'minimize' into `optimize` method
 
Use the `plot_fva` to visualize results. Store the outpus in lists corresponding to those in `plot_fva`.

In [ ]:
def plot_fva(atpm_values, max_g6pdh_values, max_pgi_values, min_g6pdh_values, min_pgi_values):
    """
    Parameters
    ----------
    atpm_values : list
        list of ATPM flux values from 0 to max
        
    max_g6pdh_values : list
        list of maximum G6PDH2r flux values at the ATPM fluxes in atpm_values
    max_pgi_values : list
        list of maximum PGI flux values at the ATPM fluxes in atpm_values

    min_g6pdh_values : list 
        list of minimum G6PDH2r flux values at the ATPM fluxes in atpm_values

    min_pgi_values : list
        list of minimum PGI flux values at the ATPM fluxes in atpm_values

    """
    plt.fill_between(atpm_values, max_g6pdh_values, min_g6pdh_values, 
                     label='G6PDH(ppp)', alpha=.7)
    plt.fill_between(atpm_values, max_pgi_values, min_pgi_values, 
                     label='PGI(glycolysis)', alpha=.7)
    plt.legend()
    plt.xlabel('ATPM flux')
    plt.ylabel('Range of possible fluxes')

### 3) Characterizing solution space (sampling)

Use the `cobra.sampling` function to sample the model 1000 times.

**Note**: this returns a dataframe. You can visualize the solutions with `df[['G6PDH2r', 'PGI']].hist()`

## C) Characteristics of core energy metabolism
### 1) Carbon yield of gluconeogenic substrats
Assess the carbon yield and gluconeogenesis pathway used to generate g6p from each carbon containing metabolite

**Approach**
1. Add exchange reaction for g6p_c to use as the objective reaction
  - Make sure the lower bound is set to 0 (i.e., g6p_c -> )

2. Set glucose uptake to zero

3. Iterate through all model metabolites
 - Skip those with no C molecules in formula
 - Can check for this using, for example, `model.metabolites.co2_c.elements.get('C', 0)`. This will return 0 if the metabolite does not contain any carbons
 
4. Check for an existing exchange reaction for the metabolite. 
 - If it does not not exist, add one
 - Set the lower bound of the exchange reaction to 1

5. Set objective to EX_g6p_c and optimize
 - Use `cobra.flux_analysis.pfba` for optimization. This will return the most parsimonious solution (i.e., the sum of flux values in the model will be minimized)


6. Calculate the carbon yield of g6p per substrate
 - Can return the number of carbons per mol of substrate using command above (i.e.,
 `mol_c_substrate = model.metabolites.co2_c.elements.get('C', 0)`)
 -  `carbon_yield_substrate = sol.fluxes['EX_g6p_c'] * 6 / mol_c_substrate`
 
7. Compute the absolute value of the flux through two mitochondrial transporters: MALtm and PEPtm
 
7. Save solution in a pandas `DataFrame` structured like below:

| Met ID      | g6p yield | MALtm |PEPtm |
| :-------------: |:-------------:|:-------------:| :-------------:|
| glc__D_c |  | |  |
| pep_c |  |   |  |
| . |  |   |  |
| . |  |   |  |

9. Use `plot_carbon_yield` to visualize solution


**Note:** make sure to either remove any newly added exchanges (or set the exchange to zero) before computing the yield for the next metabolite. You can also use the model as a context as described in https://cobrapy.readthedocs.io/en/latest/getting_started.html#Making-changes-reversibly-using-models-as-contexts 

In [ ]:
def plot_carbon_yield(input_df):
    
    fig, ax = plt.subplots(1,1)
    #glc_per_mol = glc_per_mol.rename({'glucose yield': r'G6P Yield ($\mathrm{\frac{mol\ C_{G6P}}{mol\ C_{precursor}}}$)'}, axis=1)
    ax = glc_per_mol[['MALtm', 'PEPtm']].plot(kind='bar', figsize=(12.5, 5), ax=ax)
    ax2_color = '#7c5395'
    ax2= glc_per_mol['g6p yield'].plot(secondary_y=True, color=ax2_color, linewidth=5)
    ax2.tick_params(axis='y', labelsize=15, labelcolor=ax2_color, color=ax2_color)
    ax2.set_ylabel(r'G6P Yield ($\mathrm{\frac{mol\ C_{G6P}}{mol\ C_{precursor}}}$)', c=ax2_color, 
                   rotation=-90, fontsize=25,labelpad=55)
    ax.set_ylabel(r'Transport Flux ($\mathrm{\frac{mmol}{L}}$)', fontsize=25)
    ax.legend(fontsize=15, loc='center right')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontdict={'size': 14})
    ax.tick_params(axis='y', labelsize=15)
    ax.set_title('Gluconeogenic compounds in glycolysis, PPP, and TCA', size=25)
    fig.tight_layout()
    fig

### 2) Common pitfalls: neglecting metabolite transport

Iterate through all reactions:

1. check if reactions is mitchondrial transport  using `mitochondrial_exchange`

2. if reaction is transporter, knock out

3. optimize for ATP synthesis

4. store results in dataframe and plot as bar plot

In [ ]:
def mitochondrial_exchange(reaction):
    cytosolic = False
    mitochondrial = False
    for m in reaction.metabolites:
        if m.id[-2:] == '_c':
            cytosolic = True
        if m.id[-2:] == '_m':
            mitochondrial = True
    # If reaction contains both mitochondrial and
    # cytosolic metabolite, return True
    if cytosolic and mitochondrial:
        return True
    else:
        return False